## Анализ логов
**Общая задача:** создать скрипт для формирования витрины на основе логов web-сайта.  

**Подробное описание задачи:**

Разработать скрипт формирования витрины следующего содержания:
1.      Суррогатный ключ устройства

2.      Название устройства

3.      Количество пользователей

4.      Доля пользователей данного устройства от общего числа пользователей.

5.      Количество совершенных действий для данного устройства

6.      Доля совершенных действий с данного устройства, относительно других устройств

7.      Список из 5 самых популярных браузеров, используемых на данном устройстве различными пользователями, с указанием доли использования для данного браузера относительно остальных браузеров. 

8.      Количество ответов сервера отличных от 200 на данном устройстве

9.      Для каждого из ответов сервера, отличных от 200, сформировать поле, в котором будет содержаться количество ответов данного типа

In [ ]:
pip install clickhouse_connect

In [ ]:
pip install user_agents

In [1]:
import pandas as pd
from tqdm import tqdm
from user_agents import parse
import clickhouse_connect
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_ch = pd.read_csv('./data/client_hostname.csv')
df_ch

In [2]:
%%time

file = './data/access.log'
logs = pd.DataFrame()
with open(file) as fl:
    chunk_iter = pd.read_table(fl, chunksize=100000, engine='python', header=None, index_col=False, 
                               names=['ip', 'datetime', 'request', 'code_req', 'port', 'user_agent'], 
                               sep=" - - \[{1}(.+)\] \"(.*?)\" (\d+) (\d+) .+ \"(.{2,}?)\".*")
    for chunk in chunk_iter:
        chunk = chunk[chunk['code_req'] != 200]
        logs = pd.concat([logs,chunk])
logs

CPU times: user 1min 28s, sys: 1.13 s, total: 1min 29s
Wall time: 1min 30s


,ip,datetime,request,code_req,port,user_agent
16,207.46.13.136,22/Jan/2019:03:56:19 +0330,GET /product/14926 HTTP/1.1,404.0,33617.0,Mozilla/5.0 (compatible; bingbot/2.0; +http://...
39,66.249.66.194,22/Jan/2019:03:56:23 +0330,GET /product/81900 HTTP/1.1,404.0,32278.0,Mozilla/5.0 (compatible; Googlebot/2.1; +http:...
81,5.211.97.39,22/Jan/2019:03:56:31 +0330,HEAD /amp_preconnect_polyfill_404_or_other_err...,404.0,0.0,Mozilla/5.0 (iPhone; CPU iPhone OS 10_3_2 like...
94,54.36.148.87,22/Jan/2019:03:56:34 +0330,GET /filter/p65%2Cv1%7C%D9%86%D9%82%D8%B1%D9%8...,302.0,0.0,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http:...
147,207.46.13.104,22/Jan/2019:03:56:46 +0330,GET /browse/flute-keys/www.zanbil.ir HTTP/1.1,404.0,33605.0,Mozilla/5.0 (compatible; bingbot/2.0; +http://...
...,...,...,...,...,...,...
10365121,37.129.59.160,26/Jan/2019:20:29:12 +0330,GET /basket/add/62424?mobile=1&addedValues= HT...,302.0,0.0,Mozilla/5.0 (Linux; Android 6.0.1; D6633 Build...
10365122,66.249.66.92,26/Jan/2019:20:29:12 +0330,GET /static/css/font/wyekan/font.woff HTTP/1.1,304.0,0.0,Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Bu...
10365130,5.127.220.71,26/Jan/2019:20:29:12 +0330,GET /apple-touch-icon-120x120-precomposed.png ...,404.0,32426.0,MobileSafari/604.1 CFNetwork/976 Darwin/18.2.0
10365147,188.229.21.56,26/Jan/2019:20:29:13 +0330,GET /content/view/shoppingRules HTTP/1.1,302.0,0.0,Mozilla/5.0 (Linux; Android 7.0; SM-N920C Buil...


______
### DATA QUALITY:

In [4]:
logs.isnull().sum()

ip            0
datetime      2
request       8
code_req      2
port          2
user_agent    2
dtype: int64

In [5]:
logs = logs.dropna()
print('Data is clean')

Data is clean


__________

In [6]:
logs = logs.astype({'code_req': int, 'port': int})

In [7]:
logs

,ip,datetime,request,code_req,port,user_agent
16,207.46.13.136,22/Jan/2019:03:56:19 +0330,GET /product/14926 HTTP/1.1,404,33617,Mozilla/5.0 (compatible; bingbot/2.0; +http://...
39,66.249.66.194,22/Jan/2019:03:56:23 +0330,GET /product/81900 HTTP/1.1,404,32278,Mozilla/5.0 (compatible; Googlebot/2.1; +http:...
81,5.211.97.39,22/Jan/2019:03:56:31 +0330,HEAD /amp_preconnect_polyfill_404_or_other_err...,404,0,Mozilla/5.0 (iPhone; CPU iPhone OS 10_3_2 like...
94,54.36.148.87,22/Jan/2019:03:56:34 +0330,GET /filter/p65%2Cv1%7C%D9%86%D9%82%D8%B1%D9%8...,302,0,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http:...
147,207.46.13.104,22/Jan/2019:03:56:46 +0330,GET /browse/flute-keys/www.zanbil.ir HTTP/1.1,404,33605,Mozilla/5.0 (compatible; bingbot/2.0; +http://...
...,...,...,...,...,...,...
10365121,37.129.59.160,26/Jan/2019:20:29:12 +0330,GET /basket/add/62424?mobile=1&addedValues= HT...,302,0,Mozilla/5.0 (Linux; Android 6.0.1; D6633 Build...
10365122,66.249.66.92,26/Jan/2019:20:29:12 +0330,GET /static/css/font/wyekan/font.woff HTTP/1.1,304,0,Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Bu...
10365130,5.127.220.71,26/Jan/2019:20:29:12 +0330,GET /apple-touch-icon-120x120-precomposed.png ...,404,32426,MobileSafari/604.1 CFNetwork/976 Darwin/18.2.0
10365147,188.229.21.56,26/Jan/2019:20:29:13 +0330,GET /content/view/shoppingRules HTTP/1.1,302,0,Mozilla/5.0 (Linux; Android 7.0; SM-N920C Buil...


In [8]:
%%time

logs['browser'] = logs.user_agent.apply(lambda x: parse(x).browser.family)
logs['device'] = logs.user_agent.apply(lambda x: parse(x).device.brand if parse(x).device.brand is not(None) else parse(x).os.family)
logs

CPU times: user 1min 14s, sys: 3.51 ms, total: 1min 14s
Wall time: 1min 14s


,ip,datetime,request,code_req,port,user_agent,browser,device
16,207.46.13.136,22/Jan/2019:03:56:19 +0330,GET /product/14926 HTTP/1.1,404,33617,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,bingbot,Spider
39,66.249.66.194,22/Jan/2019:03:56:23 +0330,GET /product/81900 HTTP/1.1,404,32278,Mozilla/5.0 (compatible; Googlebot/2.1; +http:...,Googlebot,Spider
81,5.211.97.39,22/Jan/2019:03:56:31 +0330,HEAD /amp_preconnect_polyfill_404_or_other_err...,404,0,Mozilla/5.0 (iPhone; CPU iPhone OS 10_3_2 like...,Mobile Safari,Apple
94,54.36.148.87,22/Jan/2019:03:56:34 +0330,GET /filter/p65%2Cv1%7C%D9%86%D9%82%D8%B1%D9%8...,302,0,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http:...,AhrefsBot,Spider
147,207.46.13.104,22/Jan/2019:03:56:46 +0330,GET /browse/flute-keys/www.zanbil.ir HTTP/1.1,404,33605,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,bingbot,Spider
...,...,...,...,...,...,...,...,...
10365121,37.129.59.160,26/Jan/2019:20:29:12 +0330,GET /basket/add/62424?mobile=1&addedValues= HT...,302,0,Mozilla/5.0 (Linux; Android 6.0.1; D6633 Build...,Chrome Mobile,Sony
10365122,66.249.66.92,26/Jan/2019:20:29:12 +0330,GET /static/css/font/wyekan/font.woff HTTP/1.1,304,0,Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Bu...,Googlebot,Spider
10365130,5.127.220.71,26/Jan/2019:20:29:12 +0330,GET /apple-touch-icon-120x120-precomposed.png ...,404,32426,MobileSafari/604.1 CFNetwork/976 Darwin/18.2.0,MobileSafari,Apple
10365147,188.229.21.56,26/Jan/2019:20:29:13 +0330,GET /content/view/shoppingRules HTTP/1.1,302,0,Mozilla/5.0 (Linux; Android 7.0; SM-N920C Buil...,Chrome Mobile,Samsung


In [9]:
logs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 785319 entries, 16 to 10365148
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   ip          785319 non-null  object
 1   datetime    785319 non-null  object
 2   request     785319 non-null  object
 3   code_req    785319 non-null  int64 
 4   port        785319 non-null  int64 
 5   user_agent  785319 non-null  object
 6   browser     785319 non-null  object
 7   device      785319 non-null  object
dtypes: int64(2), object(6)
memory usage: 53.9+ MB


In [10]:
try:
    client = clickhouse_connect.get_client(host='localhost', username='default', password='')
    print('Connected to Clickhouse server')
except:
    print ('Can not connect to Clickhouse server')
    exit()

Connected to Clickhouse server


Создание БД

In [29]:
client.command('CREATE DATABASE IF NOT EXISTS logs')
client.command('CREATE TABLE IF NOT EXISTS logs.table (ip String, datetime String, request String, code_req Integer, port Integer, user_agent String, browser String, device String) ENGINE Memory')

''

In [28]:
#client.command('DROP TABLE logs.table')

''

In [30]:
client.insert('logs.table', logs, column_names=['ip', 'datetime', 'request', 'code_req', 'port', 'user_agent', 'browser', 'device']) 

In [32]:
device_and_users_query = client.query('SELECT device, uniq(ip) AS count_users FROM logs.table GROUP BY device ORDER BY device').result_set
data_mart = pd.DataFrame(list(device_and_users_query), columns=['device_name', 'users_count'])

data_mart['percent_users'] = (data_mart['users_count'] / data_mart['users_count'].sum()) * 100

actions_query = client.query('SELECT device, count(request) AS c FROM logs.table GROUP BY device ORDER BY device').result_set
actions = pd.DataFrame(list(actions_query), columns=['device_name', 'actions_device_count'])
data_mart = data_mart.merge(actions)

data_mart['percent_actions'] = (data_mart['actions_device_count'] / data_mart['actions_device_count'].sum()) * 100

browsers_query = client.query('SELECT browser, count(*) AS c FROM logs.table GROUP BY browser ORDER BY c DESC LIMIT 6').result_set
browsers_dict = dict()
for device in data_mart.device_name:
    query = client.query("SELECT browser, count(*) AS c FROM logs.table WHERE device = '"+device+"' GROUP BY browser ORDER BY c DESC LIMIT 5").result_set
    query_dict = {}
    for y in range(len(query)):
        query_dict[query[y][0]] = query[y][1]
    browsers_dict[device] = str(query_dict)
browsers = pd.DataFrame.from_dict(browsers_dict, orient='index', columns=['top_browsers'])
browsers = browsers.reset_index()
browsers.columns = ['device_name', 'top_browsers']
data_mart = data_mart.merge(browsers)

count_codes_query = client.query('SELECT device, count(code_req) AS c FROM logs.table WHERE code_req != 200 GROUP BY device ORDER BY device').result_set
count_codes = pd.DataFrame(list(count_codes_query), columns=['device_name', 'count_code_not_200'])
data_mart = data_mart.merge(count_codes)

codes_query = client.query('SELECT device, groupArray(code_req) AS c FROM logs.table WHERE code_req != 200 GROUP BY device ORDER BY device').result_set
codes_list = list()
for i in range(len(codes_query)):
    codes_list.append((codes_query[i][0], str(dict(pd.Series(codes_query[i][1]).value_counts()))))
codes = pd.DataFrame(codes_list, columns=['device_name', 'count_code'])
data_mart = data_mart.merge(codes)


In [33]:
data_mart

,device_name,users_count,percent_users,actions_device_count,percent_actions,top_browsers,count_code_not_200,count_code
0,Acer,2,0.003286,8,0.001019,"{'Chrome': 6, 'Android': 2}",8,"{302: 7, 404: 1}"
1,Alcatel,14,0.023000,114,0.014516,"{'Android': 88, 'Chrome Mobile': 16, 'Chrome':...",114,"{404: 62, 302: 33, 499: 8, 403: 5, 500: 3, 304..."
2,Amazon,1,0.001643,1,0.000127,{'Amazon Silk': 1},1,{301: 1}
3,Apple,10377,17.048087,99326,12.647854,"{'MobileSafari': 55914, 'Mobile Safari': 26501...",99326,"{404: 69205, 302: 20520, 499: 3850, 301: 2879,..."
4,Archos,1,0.001643,6,0.000764,{'Android': 6},6,"{404: 5, 302: 1}"
5,Asus,337,0.553648,1027,0.130775,"{'Chrome Mobile': 651, 'Chrome': 175, 'Chrome ...",1027,"{302: 527, 304: 180, 499: 137, 404: 86, 403: 7..."
6,BlackBerry,14,0.023000,68,0.008659,{'BlackBerry WebKit': 68},68,"{304: 35, 404: 21, 302: 6, 499: 3, 301: 3}"
7,Chrome OS,1,0.001643,1,0.000127,{'Chrome': 1},1,{301: 1}
8,Cubot,1,0.001643,3,0.000382,{'Chrome Mobile': 3},3,"{302: 2, 499: 1}"
9,Fedora,3,0.004929,18,0.002292,"{'Firefox': 15, 'Chrome': 3}",18,"{302: 8, 499: 8, 301: 2}"


In [34]:
data_mart.to_csv('./data_mart.csv')